In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report , accuracy_score
from sklearn.model_selection import train_test_split,KFold,cross_val_score

from sklearn.model_selection import GridSearchCV

dataFrame = pd.read_csv("test.csv")
data = dataFrame
data

,class_index,review_title,review_text
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...
...,...,...,...
399995,1,Unbelievable- In a Bad Way,We bought this Thomas for our son who is a hug...
399996,1,"Almost Great, Until it Broke...",My son recieved this as a birthday gift 2 mont...
399997,1,Disappointed !!!,"I bought this toy for my son who loves the ""Th..."
399998,2,Classic Jessica Mitford,This is a compilation of a wide range of Mitfo...


In [7]:
def cross_val(model , X , y):
    kf=KFold(n_splits=5,shuffle=True)
    score=cross_val_score(model,X,y,cv=kf)
    print("Cross Validation Scores are {}".format(score))
    print("Average Cross Validation score :{}".format(score.mean()))

In [8]:
data.columns[:]

Index(['class_index', 'review_title', 'review_text'], dtype='object')

In [9]:
data.dtypes

class_index      int64
review_title    object
review_text     object
dtype: object

In [10]:
data.isnull().sum()
#data = data.dropna()

class_index      0
review_title    10
review_text      0
dtype: int64

In [6]:
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf = True , lowercase = True , strip_accents='ascii' , stop_words = stopset)

In [7]:
y=data['class_index']
y_temp = []

#XGB algoritmasında etiketlerin 0'dan başlaması gerektiğinden 1 ve 2 olan etiket değerleri 0 ve 1 olarak değiştirilmiştir.
for i in range(len(y)):
    y_temp.append(y[i]-1)
XX = vectorizer.fit_transform(data.review_text)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(XX,y_temp,test_size=0.333, random_state = 50)

# Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=1000).fit(X_train, y_train)
preds = model.predict(X_test)

print(f"The accuracy of the model is {accuracy_score(y_test,preds)*100} %")

print('Training set score: {:.6f}'.format(model.score(X_train, y_train)))

print('Test set score: {:.6f}'.format(model.score(X_test, y_test)))

print(classification_report(y_test, preds, zero_division=1, digits=6))

cross_val(model , XX , y)

### Hiper Parametre Ayarlaması

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()

param_grid = { 
     'n_estimators': [500,1000],
     'max_depth' : [5,10,15],
     'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, refit = True, cv = 3,verbose = 3)
CV_rfc.fit(X_train, y_train)

# print best parameter after tuning
print(CV_rfc.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(CV_rfc.best_estimator_)

CV_rfc_pred = CV_rfc.predict(X_test)
 
# print classification report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, f1_score
print(classification_report(y_test, CV_rfc_pred, digits=6))

#  Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_gini = DecisionTreeClassifier(max_depth=50, random_state=42)
clf_gini.fit(X_train, y_train)
y_pred_gini = clf_gini.predict(X_test)

print(f"The accuracy of the model is {accuracy_score(y_test,y_pred_gini)*100} %")

print('Training set score: {:.6f}'.format(clf_gini.score(X_train, y_train)))

print('Test set score: {:.6f}'.format(clf_gini.score(X_test, y_test)))

print(classification_report(y_test, y_pred_gini,zero_division=1,digits=6))

cross_val(clf_gini , XX , y)

### Hiper Parametre Ayarlaması

In [17]:
from sklearn.tree import DecisionTreeClassifier

dtc=DecisionTreeClassifier(random_state=42)

param_grid={#'ccp_alpha': [0.1, 0.01, 1],
            #'criterion': ['gini', 'entropy'],
            'max_depth': [50,55,60,65,70,75],
            #'max_features': ['auto', 'sqrt', 'log2'],
           },


CV_dtc = GridSearchCV(estimator=dtc, param_grid=param_grid, refit = True, verbose = 3, cv=3)
CV_dtc.fit(X_train, y_train)

print(CV_dtc.best_params_)

print(CV_dtc.best_estimator_)

CV_dtc_pred = CV_dtc.predict(X_test)

print(classification_report(y_test, CV_dtc_pred, digits=6))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END ......................max_depth=50;, score=0.735 total time= 2.7min
[CV 2/3] END ......................max_depth=50;, score=0.737 total time= 2.7min
[CV 3/3] END ......................max_depth=50;, score=0.734 total time= 3.4min
[CV 1/3] END ......................max_depth=55;, score=0.733 total time= 3.0min
[CV 2/3] END ......................max_depth=55;, score=0.735 total time= 3.0min
[CV 3/3] END ......................max_depth=55;, score=0.733 total time= 3.3min
[CV 1/3] END ......................max_depth=60;, score=0.734 total time= 3.2min
[CV 2/3] END ......................max_depth=60;, score=0.734 total time= 3.1min
[CV 3/3] END ......................max_depth=60;, score=0.733 total time= 3.1min
[CV 1/3] END ......................max_depth=65;, score=0.732 total time= 3.2min
[CV 2/3] END ......................max_depth=65;, score=0.735 total time= 3.2min
[CV 3/3] END ......................max_depth=65;,

#  K-Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3 , metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

print(f"The accuracy of the model is {accuracy_score(y_test,y_pred)*100} %")

print('Training set score: {:.6f}'.format(knn.score(X_train, y_train)))

print('Test set score: {:.6f}'.format(knn.score(X_test, y_test)))

print(classification_report(y_test, y_pred, digits=6))

cross_val(knn , XX , y)

### Hiper Parametre Ayarlaması

In [12]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier()

param_grid={'n_neighbors': [5,10,15],
            #'leaf_size': [1,10,20,30,40,50],
            'p': (1,2),
            'weights': ('uniform', 'distance'),
            'metric': ('minkowski', 'manhattan', 'chebyshev', 'euclidean')
           }

CV_knn = GridSearchCV(estimator=knn, param_grid=param_grid, refit = True, verbose = 3, cv=3)
CV_knn.fit(X_train, y_train)

print(CV_knn.best_params_)

print(CV_knn.best_estimator_)

CV_knn_pred = CV_knn.predict(X_test)

print(classification_report(y_test, CV_knn_pred, digits=6))

Fitting 3 folds for each of 48 candidates, totalling 144 fits


KeyboardInterrupt: 

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train, y_train)
y_pred = bnb.predict(X_test)

print(f"The accuracy of the model is {accuracy_score(y_test,y_pred)*100} %")

print('Training set score: {:.6f}'.format(bnb.score(X_train, y_train)))

print('Test set score: {:.6f}'.format(bnb.score(X_test, y_test)))

print(classification_report(y_test, y_pred, zero_division=1, digits=6))

cross_val(bnb , XX , y)

### Hiper Parametre Ayarlaması

In [ ]:
from sklearn.naive_bayes import BernoulliNB

bnb=BernoulliNB()

param_grid={'alpha': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10],
            'binarize': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9, 1e10],
            'fit_prior': [True, False]
           }

CV_bnb = GridSearchCV(estimator=bnb, param_grid=param_grid, refit = True, verbose = 3)
CV_bnb.fit(X_train, y_train)

print(CV_bnb.best_params_)

print(CV_bnb.best_estimator_)

CV_bnb_pred = CV_bnb.predict(X_test)

print(classification_report(y_test, CV_bnb_pred, zero_division=1, digits=6))

# XGB

In [15]:
from xgboost import XGBClassifier

model = XGBClassifier(learning_rate=0.5, max_depth=15).fit(X_train, y_train)
preds2 = model.predict(X_test)

print(f"The accuracy of the model is {accuracy_score(y_test,preds2)*100} %")

print('Training set score: {:.6f}'.format(model.score(X_train, y_train)))

print('Test set score: {:.6f}'.format(model.score(X_test, y_test)))

print(classification_report(y_test, preds2, zero_division=1, digits=6))

cross_val(model , XX , y)

              precision    recall  f1-score   support

           0   0.850658  0.843765  0.847197     66515
           1   0.845412  0.852246  0.848815     66685

    accuracy                       0.848011    133200
   macro avg   0.848035  0.848005  0.848006    133200
weighted avg   0.848032  0.848011  0.848007    133200

Cross Validation Scores are [nan nan nan nan nan]
Average Cross Validation score :nan


"\nparam = {'learning_rate':0.5, 'max_depth':15}\n\n# train with customized objective\ndtrain = xgb.DMatrix(X_train, label=y_train);\n\nxgb.cv(param, dtrain, nfold=5,\n       metrics={'auc'})"

### Hiper Parametre Ayarlaması

In [ ]:
from xgboost import XGBClassifier

gb=XGBClassifier()

param_grid={"learning_rate"    : [0.5, 0.75, 1] ,
             "max_depth"        : [10, 15, 20]
             #"min_child_weight" : [ 1, 3, 5, 7 ],
             #"gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
             #"colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] 
           }

CV_gb = GridSearchCV(estimator=gb, param_grid=param_grid, refit = True, verbose = 3, cv=3)
CV_gb.fit(X_train, y_train)

print(CV_gb.best_params_)

print(CV_gb.best_estimator_)

CV_gb_pred = CV_gb.predict(X_test)

print(classification_report(y_test, CV_gb_pred, zero_division=1, digits=6))